In [ ]:
import torch
import numpy as np
from pathlib import Path

from nemo.util.plotting import plot_surface
from nemo.field import NeuralHeightField
from nemo.dem import DEM
from nemo.nemov2 import NEMoV2

device = "cuda" if torch.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2

In [ ]:
tif_path = Path("../../data/Site01_final_adj_5mpp_surf.tif")
dem = DEM.from_file(tif_path)

In [ ]:
plot_dem = dem.downsample(10)

In [ ]:
# Load the lunar DEM data
dem = DEM.from_file("../../data/Moon_Map_01_0_rep0.dat")

# Downsample to manageable size for testing
dem_ds = dem.downsample(1)  # Reduce from 180x180 to 45x45

# Get XYZ data
xyz = dem_ds.get_xyz_combined()

# Convert to torch tensors and move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
xy = torch.from_numpy(xyz[:, :2]).float().to(device)
z = torch.from_numpy(xyz[:, 2]).float().to(device)

print(f"Real LDEM data shape: xy={xy.shape}, z={z.shape}")
print(
    f"Coordinate ranges: X({xy[:, 0].min():.3f}, {xy[:, 0].max():.3f}), Y({xy[:, 1].min():.3f}, {xy[:, 1].max():.3f})"
)
print(f"Elevation range: Z({z.min():.3f}, {z.max():.3f})")

# Create new NEMoV2 instance
nemov2 = NEMoV2(device="cuda" if torch.cuda.is_available() else "cpu")

# Test fitting with conservative settings
print("\nFitting NEMoV2 to real LDEM data...")
losses = nemov2.fit(
    xy,
    z,
    lr=1e-3,
    max_epochs=5000,
    batch_size=20000,
    verbose=False,
    early_stopping=False,
    enable_spatial=False,
)

In [ ]:
pred_z = nemov2(xy)
target_z = torch.from_numpy(dem_ds.z_coords.flatten()).to(device)

In [ ]:
pred_z, target_z

In [ ]:
import torch.nn as nn

criterion = nn.MSELoss()
loss = criterion(pred_z, target_z)
print(loss.item())

In [ ]:
pred_z = nemov2(xy)

pred_grid = dem_ds.data.copy()
pred_grid[:, :, 2] = (
    pred_z.detach().cpu().numpy().reshape(dem_ds.data.shape[0], dem_ds.data.shape[1])
)

plot_surface(pred_grid)

In [ ]:
plot_surface(dem_ds.data)

In [ ]:
plot_surface(xyz.reshape(dem_ds.data.shape))

# ---

In [ ]:
dem_path = "../../data/Moon_Map_01_0_rep0.dat"
dem = np.load(dem_path, allow_pickle=True)

In [ ]:
plot_surface(dem)

In [ ]:
# Reshape DEM data from (180, 180, 3) to (N, 3) where N = 180*180
dem_reshaped = dem[:, :, :3].reshape(-1, 3)  # Flatten spatial dimensions

# Convert to torch tensor and move to CUDA
dem_tensor = torch.from_numpy(dem_reshaped).float().to(device)

# Split into input (x,y) and target (z) tensors
xy = dem_tensor[:, :2]  # First two columns are x,y coordinates
z = dem_tensor[:, 2]  # Third column is the height value
xyz = torch.cat([xy, z.unsqueeze(1)], dim=1)

In [ ]:
field = NeuralHeightField(in_dim=2, encoding_type="nerf").to(device)

In [ ]:
field.fit(xyz, grad_weight=0.0)

In [ ]:
field_z = field(xy)
field_dem = dem.copy()
field_dem[:, :, 2] = field_z.reshape(180, 180).detach().cpu().numpy()

In [ ]:
plot_surface(field_dem)

LDEM

In [ ]:
tif_path = Path("../../data/Site01_final_adj_5mpp_surf.tif")
dem = DEM.from_file(tif_path)

In [ ]:
plot_dem = dem.downsample(10)

In [ ]:
plot_surface(plot_dem.data)

In [ ]:
import tinycudann as tcnn
from nemo.nemo import Nemo
from nemo.nemov2 import NEMoV2

In [ ]:
plot_dem.data[:, :, :2].shape

In [ ]:
nemo = NEMoV2()

xy = torch.from_numpy(plot_dem.data[:, :, :2]).reshape(-1, 2).to(device)
z = torch.from_numpy(plot_dem.data[:, :, 2]).reshape(-1, 1).to(device)

nemo.fit(xy, z, verbose=False)